In [ ]:
import pandas as pd
import numpy as np
import io

# --- 加载数据 ---
# 请将下面的StringIO替换为你的文件名，例如 'train.csv'
train_df = pd.read_csv('data/Antai_hackathon_train.csv')
attr_df = pd.read_csv('data/Antai_hackathon_attr.csv')

# 检查数据加载情况
print("--- 原始训练数据 (train.csv) ---")
print(train_df.head())
print("\n--- 原始商品属性数据 (item_attr.csv) ---")
print(attr_df.head())



--- 原始训练数据 (train.csv) ---
  buyer_country_id  buyer_admin_id  item_id    create_order_time  irank
0               xx          489859        1  2018-04-11 03:28:02      9
1               xx         2567786        2  2018-04-14 08:24:26     26
2               xx         3408746        3  2018-04-17 02:11:56      7
3               xx         2801580        4  2018-04-20 10:11:17      3
4               xx         1348149        5  2018-04-17 10:49:05      4

--- 原始商品属性数据 (item_attr.csv) ---
   item_id  cate_id  store_id  item_price
0   140446     1413     11822           1
1   403593     2313     19712           1
2   252621     1682      6622           1
3   204530     1413     11822           1
4   340076      181     24403           1


In [6]:
merged_df = pd.merge(train_df, attr_df, on='item_id', how='left')

print(train_df.shape)
print(attr_df.shape)

# 显示合并后的表格信息
print("--- 1. 数据合并结果 ---")
print("合并后表格的维度:", merged_df.shape)
print("合并后表格的前5行:")
print(merged_df.head())
print("\n")

(6989817, 5)
(1924269, 4)
--- 1. 数据合并结果 ---
合并后表格的维度: (6989817, 8)
合并后表格的前5行:
  buyer_country_id  buyer_admin_id  item_id    create_order_time  irank  \
0               xx          489859        1  2018-04-11 03:28:02      9   
1               xx         2567786        2  2018-04-14 08:24:26     26   
2               xx         3408746        3  2018-04-17 02:11:56      7   
3               xx         2801580        4  2018-04-20 10:11:17      3   
4               xx         1348149        5  2018-04-17 10:49:05      4   

   cate_id  store_id  item_price  
0   2228.0    9694.0      4491.0  
1   3667.0    4364.0      2751.0  
2    153.0    8085.0       656.0  
3   3359.0    3345.0      2501.0  
4   1156.0    1892.0       589.0  




In [7]:
print("--- 2.1 基础统计信息 ---")
# 计算独立用户、商品、品类和店铺的数量
num_users = merged_df['buyer_admin_id'].nunique()
num_items = merged_df['item_id'].nunique()
num_cates = merged_df['cate_id'].nunique()
num_stores = merged_df['store_id'].nunique()

print(f"总购买记录数: {len(merged_df)}")
print(f"独立用户数: {num_users}")
print(f"独立商品数: {num_items}")
print(f"独立品类数: {num_cates}")
print(f"独立店铺数: {num_stores}")
print("\n")

--- 2.1 基础统计信息 ---
总购买记录数: 6989817
独立用户数: 483117
独立商品数: 1852506
独立品类数: 3895
独立店铺数: 77698




In [8]:
print("--- 2.2 用户行为深度分析 ---")

# 1. 每个用户的购买次数分析
user_purchase_counts = merged_df.groupby('buyer_admin_id')['item_id'].count()
print("每个用户购买次数的描述性统计:")
print(user_purchase_counts.describe())
print("\n")

# 2. 热门商品、品类、店铺分析 (Top 10)
top_10_items = merged_df['item_id'].value_counts().head(10)
top_10_cates = merged_df['cate_id'].value_counts().head(10)
top_10_stores = merged_df['store_id'].value_counts().head(10)

print("购买次数最多的Top 10商品:")
print(top_10_items)
print("\n")

print("购买次数最多的Top 10品类:")
print(top_10_cates)
print("\n")

print("购买次数最多的Top 10店铺:")
print(top_10_stores)
print("\n")

--- 2.2 用户行为深度分析 ---
每个用户购买次数的描述性统计:
count    483117.000000
mean         14.468166
std          24.996817
min           8.000000
25%           9.000000
50%          11.000000
75%          15.000000
max       11766.000000
Name: item_id, dtype: float64


购买次数最多的Top 10商品:
item_id
7493101    31625
516873      9295
3964264     9006
7557509     7253
7482805     7078
3058048     6097
426022      5627
1872817     3770
413606      3672
6162363     3585
Name: count, dtype: int64


购买次数最多的Top 10品类:
cate_id
563.0     341506
2.0       210655
1819.0    198756
2300.0    150974
1207.0    111600
1817.0    109585
1530.0     93616
584.0      91689
1512.0     80863
7.0        77505
Name: count, dtype: int64


购买次数最多的Top 10店铺:
store_id
10541.0    69534
63651.0    54106
3426.0     32061
2750.0     27761
29824.0    17740
645.0      16364
48458.0    15125
1801.0     14912
26092.0    13255
1.0        12932
Name: count, dtype: int64




In [9]:
print("--- 2.3 时间规律分析 ---")

# 首先，将时间字符串转换为datetime对象，方便进行时间分析
merged_df['create_order_time'] = pd.to_datetime(merged_df['create_order_time'])

# 提取星期几(0=Mon, 6=Sun)和小时
merged_df['day_of_week'] = merged_df['create_order_time'].dt.dayofweek
merged_df['hour_of_day'] = merged_df['create_order_time'].dt.hour

# 1. 按星期几分析购买量
purchases_by_dow = merged_df['day_of_week'].value_counts().sort_index()
print("按星期几分布的购买量 (0=周一, 6=周日):")
print(purchases_by_dow)
print("\n")

# 2. 按小时分析购买量
purchases_by_hour = merged_df['hour_of_day'].value_counts().sort_index()
print("按小时分布的购买量:")
print(purchases_by_hour)
print("\n")

# 3. 数据集时间跨度
min_time = merged_df['create_order_time'].min()
max_time = merged_df['create_order_time'].max()
print(f"数据记录的时间范围: 从 {min_time} 到 {max_time}")
print("\n")

--- 2.3 时间规律分析 ---
按星期几分布的购买量 (0=周一, 6=周日):
day_of_week
0    1021491
1    1042842
2    1058226
3    1049259
4    1055864
5     980004
6     782131
Name: count, dtype: int64


按小时分布的购买量:
hour_of_day
0     391727
1     392134
2     379994
3     379165
4     386147
5     383304
6     376736
7     364336
8     365594
9     387881
10    416622
11    431634
12    413292
13    331125
14    212999
15    125788
16     77711
17     62377
18     71766
19    100379
20    143258
21    191982
22    265408
23    338458
Name: count, dtype: int64


数据记录的时间范围: 从 2018-03-13 04:01:00 到 2018-04-28 23:59:57


